In [32]:
import json
import spacy
import re
import difflib as dl
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
def read_award_names_from_file(file):
  with open(file) as f:
    award_names = f.readlines()
  for i in range(len(award_names)):
    award_names[i] = award_names[i][:-1]
  return award_names

In [34]:
def clean_award_name(award_name):
  mystop_words = stopwords.words('english')
  mystop_words.extend(['performance','made','-','congrats','congratulations','got','knows','well','well-','hell','yeah','go'])
  #removes stopwords from award name and returns key words in award name as list
  award_name = award_name.lower()
  cleaned_award = []
  split = award_name.split()
  for word in split:
    if word not in mystop_words and word not in ['motion','picture','series,','series','role']:  # Many words are irrelevant, removed with list of stopwords
      cleaned_award.append(word)
      if word == 'television':  # Usually television is shortened to tv, want to account for both
        cleaned_award.append('tv')
  return cleaned_award

In [35]:
def open_jsonfile(data_path):
  file_ = open(data_path)
  data = json.load(file_)
  return data

In [36]:
def award_relevant_tweets(tweet_data, award_keywords):
  relevant_tweets = []
  for tweet in tweet_data:
    text = tweet['text']
    text = text.lower()
    tweet_text = tweet['text']
    tweet_text.replace('-',' ')
    tweet_text.replace('/',' ')
    if 'rt @' in text:
      continue
    if 'tv' in award_keywords and not re.search('tv|television|series',text):
      continue
    if 'tv' not in award_keywords and re.search('tv|television|series',text):
      continue
    check = 0
    for key_word in award_keywords:
      if key_word not in text:
        check +=1
    if 'tv' in award_keywords:
      if check <2:
        relevant_tweets.append(tweet_text)
    else:
      if check ==0:
        relevant_tweets.append(tweet_text)
  return relevant_tweets

In [37]:
presenter_key_words = 'present|intro|announce|gave|presenters|presenter|presenting|introducing'
award_names = read_award_names_from_file('/content/drive/MyDrive/NLP/award_names.txt')
cleaned_award_names = []
for i in range(len(award_names)):
  cleaned_award_names.append(clean_award_name(award_names[i]))
tweet_data = open_jsonfile('/content/drive/MyDrive/NLP/gg2013.json')

for award in cleaned_award_names:
  hosts = {}
  award_tweets = award_relevant_tweets(tweet_data,award)
  for tweet_text in award_tweets:
    presenter_key_words_index = re.search(presenter_key_words,tweet_text.lower())
    if presenter_key_words_index:
      doc = nlp(tweet_text[:presenter_key_words_index.start()])
      for entity in doc.ents:
        if entity.label_ == 'PERSON':
          if entity.text in hosts.keys():
            hosts[entity.text] = hosts[entity.text] + 1
          else:
            hosts[entity.text] = 1

  hosts = dict(sorted(hosts.items(),key = lambda host:host[1],reverse=True))
  if (len(hosts.keys())!=0):
    print(award,end= ' : ')
    print(list(hosts.keys())[:2])
    print()

['best', 'screenplay'] : ['Robert Pattinson', 'Amanda Seyfried']

['best', 'director'] : ['Halle Berry', 'Ben Affleck - Argo']

['best', 'foreign', 'language', 'film'] : ['Sylvester Stallone', 'Arnold Schwarzenegger']

['best', 'actor', 'supporting'] : ['Bradley Cooper', 'Kate Hudson']

['best', 'actress', 'supporting', 'mini-series', 'television', 'tv'] : ['Dennis Quaid', 'Kerry Washington']

['best', 'comedy', 'musical'] : ['GoldenGlobes', 'Will Ferrell']

['best', 'actress', 'comedy', 'musical'] : ['GoldenGlobes', 'Will Ferrell']

['best', 'mini-series', 'television', 'tv'] : ['Dennis Quaid', 'Kerry Washington']

['best', 'original', 'score'] : ['Toronto']

['best', 'actress', 'television', 'tv', 'drama'] : ['Lea Michele']

['best', 'drama'] : ['GoldenGlobes', 'Bradley Cooper']

['best', 'actress', 'supporting'] : ['Anne Hathaway', 'Robert Pattinson']

['best', 'television', 'tv', 'drama'] : ['Lea Michele']

['best', 'actress', 'mini-series', 'television', 'tv'] : ['Dennis Quaid', '